In [ ]:
#include <Wire.h>
#include <LiquidCrystal_I2C.h>

// Set the LCD dimensions (16x2)
LiquidCrystal_I2C lcd(0x27, 16, 2);



#include <WiFi.h>
#include "time.h"
#include <WiFiClient.h>
#include <BlynkSimpleEsp32.h>

// Your WiFi credentials.
// Set password to "" for open networks.
char ssid[] = "meter";
char pass[] = "12121212";


unsigned long previousMillis = 0;  // Variable to store the previous time
const long interval = 20000;        // Interval in milliseconds (20 second)

#include "ACS712.h"
#include <ZMPT101B.h>
#include <Wire.h>
#include <Adafruit_SSD1306.h>

ACS712 ACS(34, 3.7, 4095, 100);  //  ACS712 current sensor module, 34 pin no ,3.7 v referrence for volatge sensor, 4095 Analog-to-Digital Converter, 100 scalling factor
ZMPT101B voltageSensor(35, 50.0);  // ZMPT101B voltage sensor, 35 pin no , nominal AC frequency in Hz

#define SCREEN_WIDTH 128
#define SCREEN_HEIGHT 64
#define OLED_RESET -1

Adafruit_SSD1306 display(SCREEN_WIDTH, SCREEN_HEIGHT, &Wire, OLED_RESET);

float kwh;

int v, c, w;


#define switchOne 5//output
#define switchTwo 18
#define switchThree 19

#define switchOneIn 13 //input
#define switchTwoIn 12
#define switchThreeIn 14



#define buzzer 4



int lastButtonStateOne = 1;    // previous state of the button
int lastButtonStateTwo = 1;    // previous state of the button
int lastButtonStateThree = 1;  // previous state of the button


bool one = true;
bool two = true;
bool three = true;

#define Gas 33 //pin33


void setup() {
  Serial.begin(115200);

  // Configure pin 33 as an input for the gas sensor
  pinMode(Gas, INPUT);

  pinMode(switchOne, OUTPUT);
  pinMode(switchTwo, OUTPUT);
  pinMode(switchThree, OUTPUT);

  pinMode(buzzer, OUTPUT);

  digitalWrite(switchOne, HIGH);
  digitalWrite(switchTwo, HIGH);
  digitalWrite(switchThree, HIGH);


  pinMode(switchOneIn, INPUT);
  pinMode(switchTwoIn, INPUT);
  pinMode(switchThreeIn, INPUT);

  Serial.println(__FILE__);
  Serial.print("ACS712_LIB_VERSION: ");
  Serial.println(ACS712_LIB_VERSION);

  ACS.autoMidPoint();
  Serial.print("MidPoint: ");
  Serial.println(ACS.getMidPoint());
  Serial.print("Noise mV: ");
  Serial.println(ACS.getNoisemV());

  voltageSensor.setSensitivity(500.0f);


  lcd.begin();
  lcd.backlight();  // Turn on the backlight (if available on your display)


  // Print a message on the LCD
  lcd.setCursor(0, 0);  // Set the cursor to the first column and first row
  lcd.print("IoT Energy Meter");



}

void loop() {


  // Read the analog value from pin 33
  int sensorValue = analogRead(33);

  // Print the analog value to the Serial Monitor
  Serial.print("Gas Sensor Value: ");
  Serial.println(sensorValue);

  if (sensorValue > 500) {
    Blynk.logEvent("alert", "Gas Detect");

    lcd.clear();

    // Print a message on the LCD
    lcd.setCursor(1, 1);  // Set the cursor to the first column and first row
    lcd.print("Gas Detected");


    // Make the buzzer beep 5 times
    for (int i = 0; i < 5; i++) {
      digitalWrite(buzzer, HIGH);  // Turn the buzzer on
      delay(200);                  // Wait for 200 milliseconds
      digitalWrite(buzzer, LOW);   // Turn the buzzer off
      delay(200);                  // Wait for 200 milliseconds
    }

    // Add a delay before repeating (if needed)
    delay(2000);  // Wait for 2 seconds before looping again (optional)
  }



  int buttonStateOne = digitalRead(switchOneIn);
  if (buttonStateOne == HIGH) {
    digitalWrite(buzzer, HIGH);
    delay(200);
    digitalWrite(buzzer, LOW);
  }
  // Check for a button press
  if (buttonStateOne != lastButtonStateOne) {
    if (buttonStateOne == LOW) 
    {
        //delay(10);
      Serial.println("Button Click");

      if (one == true) 
      {
        Serial.println("LOW");
        one = false;
        digitalWrite(switchOne, HIGH); //This is a function that sets the voltage level of a specified digital pin.
        
      } 
      else 
      {
        Serial.println("HIGH");
        one = true;
        digitalWrite(switchOne, LOW);
        
      }
    }
    delay(10);  // Debounce delay
  }
  lastButtonStateOne = buttonStateOne;



  int buttonStateTwo = digitalRead(switchTwoIn);
  if (buttonStateTwo == HIGH) {
    digitalWrite(buzzer, HIGH);
    delay(200);
    digitalWrite(buzzer, LOW);
  }
  // Check for a button press
  if (buttonStateTwo != lastButtonStateTwo) {
    if (buttonStateTwo == LOW) {
      // delay(10);
      Serial.println("Button Click");
      if (two == true) {
        Serial.println("LOW");
        two = false;
        digitalWrite(switchTwo, HIGH);
        
      } else {
        Serial.println("HIGH");
        two = true;
        digitalWrite(switchTwo, LOW);
        
      }
    }
    delay(10);  // Debounce delay
  }
  lastButtonStateTwo = buttonStateTwo;






  int buttonStateThree = digitalRead(switchThreeIn);
  if (buttonStateThree == HIGH) {
    digitalWrite(buzzer, HIGH);
    delay(200);
    digitalWrite(buzzer, LOW);
  }
  // Check for a button press
  if (buttonStateThree != lastButtonStateThree) {
    if (buttonStateThree == LOW) {
      //delay(10);
      Serial.println("Button Click");
      if (three == true) {
        Serial.println("LOW");
        three = false;
        digitalWrite(switchThree, HIGH);
        
      } else {
        Serial.println("HIGH");
        three = true;
        digitalWrite(switchThree, LOW);
        
      }
    }
    delay(10);  // Debounce delay
  }
  lastButtonStateThree = buttonStateThree;






  
  float average = 0;
  for (int i = 0; i < 100; i++) {
    average += ACS.mA_AC();
  }
  float mA = average / 100.0;


    c = mA/1000;
    Serial.print("Current: ");
    Serial.println(c);


  
  float voltage = voltageSensor.getRmsVoltage();

    v = voltage;

    Serial.print("voltage: ");
    Serial.println(v);
 


    double watt = voltage * (mA / 1000.0);
    w = watt;


    Serial.print("watt: ");
    Serial.println(w);
    double kWh = watt / (3600*60);
    kwh = kwh + kWh;
  

  delay(1000);



  unsigned long currentMillis = millis();  // Get the current time

  // Check if the time interval has passed
  if (currentMillis - previousMillis >= interval) {
    // Update the previous time for the next interval
    previousMillis = currentMillis;

    if (kwh >= 0.05) {
      
      for (int i = 0; i < 5; i++) {
        digitalWrite(buzzer, HIGH);  // Play tone at 1000 Hz
        delay(200);                  // Duration of the tone in milliseconds
        digitalWrite(buzzer, LOW);   // Stop the tone
        delay(200);                  // Delay between tones
      }

      //delay(20000);
    }

    String tk;
    String kwhS;

    // Your code to execute every interval goes here
    
    if (kwh > 0) {
      Serial.print("kWh: ");
      Serial.println(kwh);
      kwhS = String(kwh) + "kWh";
      tk = String(kwh * 7) + "Tk";
    } else {
      tk = "0.00Tk";
      kwhS = "0.00kWh";
    }

    lcd.clear();


    int quotient = c / 1000;   // Get the integer part of the quotient
    int remainder = c % 1000;  // Get the remainder

    // Extract the first two digits of the remainder
    int firstDecimal = remainder / 100;         // Get the first digit after the decimal
    int secondDecimal = (remainder / 10) % 10;  // Get the second digit after the decimal

    // Format the result to always show two decimal places
    String Am = String(quotient) + "." + String(firstDecimal) + String(secondDecimal);

    // Output the result
    Serial.println(A);  



    // Print a message on the LCD
    lcd.setCursor(0, 0);  // Set the cursor to the first column and first row
    lcd.print("U:");
    lcd.print(kwhS);
    lcd.print(" ");
    lcd.setCursor(11, 0);
    lcd.print("kwh");


    lcd.setCursor(0, 1);  // Set the cursor to the first column and first row
    lcd.print("B:");
    lcd.print(tk);
    lcd.print(" ");
    lcd.setCursor(12, 1);
    lcd.print(v);
    lcd.print("V");
  }
}
